In [1]:
import csv
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd

In [2]:
img_file = []
text = []
label = []

with open('../Data/text_processed.csv', 'r') as f:
    reader = csv.reader(f, delimiter=',')
    header = 0
    for row in reader: 
        text.append((row[0],row[1]))
        label.append(row[2])
        
text = text[1:]
label = label[1:]

In [5]:
# Splitting train and test data
text_train, text_test, label_train,label_test = train_test_split(text, label)

print("Train Set")
print("SPAM:", label_train.count("Spam"))
print("HAM:", label_train.count("Ham"))
print("\n")
print("Test Set")
print("SPAM:", label_test.count("Spam"))
print("HAM:", label_test.count("Ham"))

Train Set
SPAM: 1459
HAM: 284


Test Set
SPAM: 498
HAM: 84


In [6]:
train_set = open("../Data/text_train.csv", "a+", encoding="utf-8")
train_set.write("Image_File,Train_Text,IsSpam\n")

for (train_t,train_l) in zip(text_train, label_train):
    train = f"{train_t[0]},{train_t[1]},{train_l}\n"
    train_set.write(train)

In [7]:
# Run count vectorizer
vectorizer = CountVectorizer()

In [8]:
csv_file = open("../Data/text_train.csv", mode='r', encoding="utf-8")
df = pd.read_csv(csv_file)
df.head()

,Image_File,Train_Text,IsSpam
0,../Data/Processed\SpamHunter Spam\2618.jpg,dull daucare eoepeli s ponday olb apr ps infor...,Spam
1,../Data/Processed\Dredze Spam\731.jpg,investor alert issue qcpc jan 30th gcpc quante...,Spam
2,../Data/Processed\Dredze Ham\380.jpg,spend 27 million get lousy tshirt ef ahi,Ham
3,../Data/Processed\Dredze Spam\832.jpg,investor alert issue start right eye glue mond...,Spam
4,../Data/Processed\Dredze Spam\1121.jpg,fix low rate mortgage notice preapproved 42000...,Spam


In [9]:
# matrix = vectorizer.fit_transform(df['Train_Text'])
matrix = vectorizer.fit_transform(df['Train_Text'].values.astype('U'))
matrix 

<1737x24056 sparse matrix of type '<class 'numpy.int64'>'
	with 94088 stored elements in Compressed Sparse Row format>

In [10]:
counts = pd.DataFrame(matrix.toarray(),columns=vectorizer.get_feature_names_out())
counts

,00,000,00000000,00005,00011,0003,00055,0006,0007,0008,...,ztnhertance,zu,zum,zumerfolg,zurich,zusammenarbeit,zy,zyban,zzle,zzpropertyref
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1732,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1733,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1734,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1735,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
# Join feature columns with original datafram
df_text = df.pop('Train_Text')
df_label = df.pop('IsSpam')
counts.insert(0, "Train_Text", df_text, True)
counts['IsSpam'] = df_label

counts.head()

,Train_Text,00,000,00000000,00005,00011,0003,00055,0006,0007,...,zu,zum,zumerfolg,zurich,zusammenarbeit,zy,zyban,zzle,zzpropertyref,IsSpam
0,dull daucare eoepeli s ponday olb apr ps infor...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Spam
1,investor alert issue qcpc jan 30th gcpc quante...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Spam
2,spend 27 million get lousy tshirt ef ahi,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Ham
3,investor alert issue start right eye glue mond...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Spam
4,fix low rate mortgage notice preapproved 42000...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Spam


In [12]:
counts.to_csv('../Data/feature_extract.csv')